### 1. Feladat- Tartalomalapú ajánlórendszer MovieLens adatokon

A MovieLens adatbázis a felhasználók filmértékeléseit és a filmek jellemzőit (műfaj, leírás, szereplők) tartalmazza.
Készíts egy tartalomalapú ajánlórendszert, amely a filmek jellemzői alapján ad ajánlásokat!

Teendők:

- Töltsd be a MovieLens adatokat (ratings és movies táblák).
- Ellenőrizd, hogy a két tábla movieID alapján összekapcsolható.
- Hozz létre egy TF-IDF + Cosine Similarity alapú modellt, amely:
- a genre (műfaj) oszlop alapján hasonlítja össze a filmeket, és megadja, mely filmek hasonlítanak legjobban egy kiválasztott filmhez.

Írj függvényt: recommend_movies(movie_title), ami 3 hasonló filmet ajánl.

Tipp: próbáld ki a „Toy Story (1995)” és „Star Wars (1977)” filmekre!

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



rating_colnames = ['userID',
                   'movieID',
                   'unix_timestamp',
                   'rating'
                   ]

ratings = pd.read_csv('https://raw.githubusercontent.com/rfarkas/student_data/main/MovieLens/movielens.small.ratings', sep='\t', names=rating_colnames)

item_colnames = [
    'movieID',
    'title',
    'genre',
    'plot',
    'actors'
]


movies = pd.read_csv(
    'https://raw.githubusercontent.com/rfarkas/student_data/main/MovieLens/movielens.item.info',
    sep='\t',
    names=item_colnames,
    encoding='latin-1'
)

print(movies.head())
print(ratings.head())


lens = pd.merge(ratings, movies, on='movieID', how='left')
lens.head()


vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(movies['genre'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend_movies(movie_title, cosine_sim=cosine_sim):

    idx = movies.index[movies['title'] == movie_title][0]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:4]

    movie_indices = [i[0] for i in sim_scores]

    return movies.iloc[movie_indices][['title', 'genre']]


recommended_movies = recommend_movies('Star Wars (1977)')
display(recommended_movies)
recommended_movies = recommend_movies('Toy Story (1995)')
display(recommended_movies)

   movieID              title                        genre  \
0        1   Toy Story (1995)  Animation,Children's,Comedy   
1        2   GoldenEye (1995)    Action,Adventure,Thriller   
2        3  Four Rooms (1995)                     Thriller   
3        4  Get Shorty (1995)          Action,Comedy,Drama   
4        5     Copycat (1995)         Crime,Drama,Thriller   

                                                plot  \
0  A little boy named Andy loves to be in his roo...   
1  When a deadly satellite weapon system falls in...   
2  This movie features the collaborative director...   
3  Some guys get all the luck, whether they like ...   
4  Criminal profiler and psychologist Dr Helen Hu...   

                                              actors  
0  Bradley, Lisa (I);Derryberry, Debi;Freeman, Sa...  
1  Arthur, Michelle (I);Bond, Samantha (I);Dench,...  
2  Beals, Jennifer;Bellais, Monica Lee (I);Blair,...  
3  Arthur, Rebeca (I);Bega, Leslie;Burke, Carleas...  
4  Amos, Diane;

,title,genre
180,Return of the Jedi (1983),"Action,Adventure,Romance,Sci-Fi,War"
171,The Empire Strikes Back (1980),"Action,Adventure,Drama,Romance,Sci-Fi,War"
270,Starship Troopers (1997),"Action,Adventure,Sci-Fi,War"


,title,genre
421,Aladdin and the King of Thieves (1996),"Animation,Children's,Comedy"
101,The Aristocats (1970),"Animation,Children's"
403,Pinocchio (1940),"Animation,Children's"


### 2. Feladat – TF-IDF helyett Word2Vec alapú tartalmi ajánló

A TF-IDF figyelmen kívül hagyja a szavak sorrendjét és kontextusát.
Készíts egy Word2Vec-alapú tartalomalapú ajánlót, amely figyelembe veszi a műfajok közötti szemantikai hasonlóságot is!

Teendők:
- Tokenizáld a genre mezőt (pl. Action, Drama, Thriller → lista formában).
- Használd a gensim Word2Vec modelljét a műfajok vektorizálásához.
- Számítsd ki a filmek közti átlagos vektoriális hasonlóságot (cosine similarity).
- Hasonlítsd össze az eredményeket a TF-IDF-es modellel.

Kérdés: A Word2Vec vagy a TF-IDF ad pontosabb ajánlást? Miért?

In [23]:
import numpy as np
#import gensim.downloader as api

### 3. Feladat – Hibrid ajánlórendszer

Kombináld a kollaboratív szűrést és a tartalomalapú ajánlást egyetlen hibrid modellbe!
A cél: egyszerre vegyük figyelembe a filmek tartalmi hasonlóságát és a felhasználói viselkedést.

Teendők:
- Számítsd ki az Item-based CF mátrixot (Pearson korrelációval).
- Készíts egy TF-IDF alapú tartalmi hasonlósági mátrixot (genre alapján).
- Kombináld a kettőt egy súlyozott hibrid mátrixba:
    - alpha = 0.6  hybrid_sim = alpha * item_sim_cf.fillna(0) + (1 - alpha) * item_sim_content.fillna(0)
    - Írj függvényt, ami a hibrid mátrix alapján ajánl hasonló filmeket.
    - Vizsgáld meg, hogyan változik az ajánlás, ha az alpha értéke 0.3, 0.5, 0.8.

Haladó ötlet: Ábrázold a hibrid hasonlósági mátrixot hőtérképen (seaborn.heatmap).

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

ratings_matrix = ratings.pivot(index='userID', columns='movieID', values='rating')

item_similarity = ratings_matrix.T.corr(method="pearson")

item_sim_df = pd.DataFrame(
                            item_similarity,
                            index=ratings_matrix.columns,
                            columns=ratings_matrix.columns
                        )



alpha = 0.8
hybrid_sim = alpha * item_similarity.fillna(0) + (1 - alpha) * item_sim_df.fillna(0)

def hybrid_recommend(movie_title, top_n=5):

    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieID'].values[0]

    sim_scores = hybrid_sim[movie_id].sort_values(ascending=False)

    similar_ids = sim_scores.index[1:top_n+1]

    return movies[movies['movieID'].isin(similar_ids)][['title', 'genre']]


recommendations = hybrid_recommend('Star Wars')
display(recommendations)



#sns.heatmap(recommendations, annot=True, cmap="coolwarm", fmt=".2f")
#plt.show()

,title,genre
49,Star Wars (1977),"Action,Adventure,Romance,Sci-Fi,War"
721,Nine Months (1995),Comedy
825,The Phantom (1996),Adventure
971,Passion Fish (1992),Drama
976,The Substitute (1996),Action


## 4. Extra Feladat – Hibrid modell bővítése NLP-vel

Ha szeretnéd kipróbálni a modern szöveges modelleket:
- A plot (filmtartalom-leírás) oszlopot vektorizáld Sentence-BERT vagy OpenAI Embedding segítségével.
- Ezt kombináld a felhasználói viselkedési adatokkal (CF).
- Elemezd, mennyiben ad pontosabb vagy emberibb ajánlásokat a modell.

Kérdés: A szöveges leírások beemelése javítja vagy rontja az ajánlás pontosságát?